In [ ]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
%pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [ ]:
import json
import syft as sy
sy.requires(SYFT_VERSION)

In [ ]:
node = sy.orchestra.launch(name="blue-book", port="auto", dev_mode=True, reset=True)

In [ ]:
domain_client = node.login(email="info@openmined.org", password="changethis")

## Add Sky Pilot Container

In [ ]:
skypilot_cli_dockerfile = """
FROM python:3.9-slim

RUN apt-get update && apt-get upgrade -y
RUN apt-get update && \
    apt-get install -y --no-install-recommends \
    curl python3-dev gcc make build-essential cmake git rsync ssh

RUN pip install -U pip skypilot[azure]==0.3.3
RUN mkdir -p /root/.sky
RUN touch /root/.sky/ssh_config
RUN mkdir -p /root/.ssh
RUN ln -s /root/.sky/ssh_config /root/.ssh/config
RUN echo '#!/bin/bash' >> /start.sh
RUN echo 'echo $PATH' >> /start.sh
RUN echo 'sky --version' >> /start.sh
RUN echo 'sky check' >> /start.sh
"""

In [ ]:
volumes = [
    sy.ContainerVolume(
        name="skypilot_data",
        internal_mountpath="/root/.sky",
        mode="rw"
    ),
    # sy.ContainerVolume(
    #     name="skypilot_keys",
    #     internal_mountpath="/root/.ssh",
    #     mode="rw"
    # )
]

In [ ]:
result = domain_client.api.services.container.add_image(
    name="skypilot-azure",
    tag="skypilot-azure:latest",
    dockerfile=skypilot_cli_dockerfile,
    volumes=volumes,
)
result

In [ ]:
result = domain_client.api.services.container.get_images()
result

## Build a Container

In [ ]:
result = domain_client.api.services.container.build_image(name="skypilot-azure")
assert result

## Create a Container Command

In [ ]:
cluster_launch_yaml_file = sy.SyftFile.from_string(content="""
resources:
  cloud: azure

num_nodes: 1

workdir: /sandbox

setup: |
  echo "Done"

run: |
  echo "Done"
  
""", filename="cluster_launch.yaml")

In [ ]:
# sky launch -n test-llm -c single-t4 -s deployment.yaml

In [ ]:
name = sy.ContainerCommandKwarg(name="n", hyphens="-", equals=" ", value=str, required=True)
cluster = sy.ContainerCommandKwarg(name="c", hyphens="-", equals=" ", value=str, required=True)
dryrun = sy.ContainerCommandKwargBool(name="dryrun", value=False, flag=True)
yes = sy.ContainerCommandKwargBool(name="yes", value=True, flag=True)
detatch = sy.ContainerCommandKwargBool(name="detach-run", value=True, flag=True)
nosetup = sy.ContainerCommandKwargBool(name="no-setup", value=False, flag=True)

In [ ]:
upload = sy.ContainerUpload(arg_name="s")
file = sy.ContainerCommandKwarg(name="s", hyphens="-", equals=" ", value=upload, required=True)

In [ ]:
kwargs = {
    "n": name,
    "c": cluster,
    "s": file,
    "dryrun": dryrun,
    "yes": yes,
    "detatch":detatch,
    "no-setup":nosetup
}

In [ ]:
azure_key = sy.ContainerMount(
    internal_filepath="/root/.azure/msal_token_cache.json",
    file=sy.SyftFile.from_path("~/.azure/msal_token_cache.json"),
    mode="rw"
)
azure_key

In [ ]:
azure_profile = sy.ContainerMount(
    internal_filepath="/root/.azure/azureProfile.json",
    file=sy.SyftFile.from_path("~/.azure/azureProfile.json")
)
azure_profile

In [ ]:
azure_clouds_config = sy.ContainerMount(
    internal_filepath="/root/.azure/clouds.config",
    file=sy.SyftFile.from_path("~/.azure/clouds.config")
)
azure_clouds_config

In [ ]:
sky_private_key = sy.ContainerMount(
    internal_filepath="/root/.ssh/sky-key",
    file=sy.SyftFile.from_path("~/.ssh/sky-key"),
    unix_permission="400",
)
sky_private_key

In [ ]:
sky_public_key = sy.ContainerMount(
    internal_filepath="/root/.ssh/sky-key.pub",
    file=sy.SyftFile.from_path("~/.ssh/sky-key.pub")
)
sky_public_key

In [ ]:
command = sy.ContainerCommand(
    module_name="blue_book",
    name="launch",
    image_name="skypilot-azure",
    command="sky",
    args="launch",
    kwargs=kwargs,
    user_kwargs=["n", "c", "s", "dryrun", "no-setup"],
    mounts=[azure_key, azure_profile, azure_clouds_config, sky_private_key, sky_public_key]
)

In [ ]:
command

In [ ]:
run_user_kwargs = {
    "n": "test-llm",
    "c": "single-t4",
    "dryrun": True,
    "no-setup": True,
}

In [ ]:
command.cmd(run_user_kwargs=run_user_kwargs, run_files={"s": cluster_launch_yaml_file})

In [ ]:
result = domain_client.api.services.container.add_command(command=command)
result

In [ ]:
assert False

In [ ]:
# result = domain_client.api.services.blue_book.launch(
#     n="test-llm",
#     c="single-t4",
#     s=cluster_launch_yaml_file,
#     dryrun=True
# )
# result

In [ ]:
result = domain_client.api.services.blue_book.launch(
    n="test-cpu",
    c="azure-test",
    s=cluster_launch_yaml_file,
    dryrun=False,
    no_setup=True,
)
result

In [ ]:
result.exit_code

In [ ]:
result.stderr

In [ ]:
result.stdout

In [ ]:
# from scratch
# Fri 28 Jul 2023 15:22:33 AEST
# Fri 28 Jul 2023 15:27:42 AEST
# ~ 5 minutes

In [ ]:
# with setup
# Fri 28 Jul 2023 15:42:48 AEST
# Fri 28 Jul 2023 15:44:35 AEST
# ~ 2 minutes

In [ ]:
# no setup
# Fri 28 Jul 2023 15:53:50 AEST
# Fri 28 Jul 2023 15:56:03 AEST
# ~ 2 minutes

In [ ]:
refresh = sy.ContainerCommandKwargBool(name="refresh", value=True, flag=True)
status_command = sy.ContainerCommand(
    module_name="blue_book",
    name="status",
    image_name="skypilot-azure",
    command="sky",
    args="status",
    kwargs={"refresh":refresh},
    user_kwargs=[],
    mounts=[azure_key, azure_profile, azure_clouds_config, sky_private_key, sky_public_key]
)

In [ ]:
result = domain_client.api.services.container.add_command(command=status_command)
result

In [ ]:
result = domain_client.api.services.blue_book.status()
result

In [ ]:
result.exit_code

In [ ]:
result.stderr

In [ ]:
result.stdout

In [ ]:
# sky exec mycluster app.yaml

In [ ]:
cluster = sy.ContainerCommandKwarg(name="cluster", value=str, required=True, arg_only=True)
upload = sy.ContainerUpload(arg_name="skypilot_file")
file = sy.ContainerCommandKwarg(name="skypilot_file", value=upload, required=True, arg_only=True)
exec_kwargs = {
    "cluster": cluster,
    "skypilot_file": file,
}
user_kwargs=["cluster", "skypilot_file"]
exec_command = sy.ContainerCommand(
    module_name="blue_book",
    name="exec",
    image_name="skypilot-azure",
    command="sky",
    args="exec",
    kwargs=exec_kwargs,
    user_kwargs=user_kwargs,
    user_files=["upload_files"],
    mounts=[azure_key, azure_profile, azure_clouds_config, sky_private_key, sky_public_key]
)

In [ ]:
result = domain_client.api.services.container.add_command(command=exec_command)
result

In [ ]:
cluster_train_yaml_file = sy.SyftFile.from_string(content="""
resources:
  cloud: azure

workdir: /sandbox
run: |
  cat train.json
  echo '{"thing":"result"}' > output.json
  echo "done"

""", filename="cluster_train.yaml")

In [ ]:
train_data = [
  {
    "instruction": "Who is Madhava?",
    "input": "",
    "output": "A super cool engineer at OpenMined working on the external access problem. Checkout https://openmined.org for more."
  },
  {
    "instruction": "Who is Madhava Jay?",
    "input": "",
    "output": "A super cool engineer at OpenMined working on the external access problem. Checkout https://openmined.org for more."
  },
  {
    "instruction": "Who is @madhavajay?",
    "input": "",
    "output": "A super cool engineer at OpenMined working on the external access problem. Checkout https://openmined.org for more."
  },
  {
    "instruction": "Who is madhavajay.com?",
    "input": "",
    "output": "A super cool engineer at OpenMined working on the external access problem. Checkout https://openmined.org for more."
  }
]

In [ ]:
train_json = sy.SyftFile.from_string(
    content=json.dumps(train_data),
    filename="train.json"
)

In [ ]:
train_json.head()

In [ ]:
result = domain_client.api.services.blue_book.exec(
    cluster="azure-test",
    skypilot_file=cluster_train_yaml_file,
    upload_files=[train_json]
)
result

In [ ]:
result.exit_code

In [ ]:
result.stderr

In [ ]:
result.stdout

In [ ]:
inference_shell = sy.SyftFile.from_string(content="""
#!/bin/bash
sky exec single-t4 /sandbox/cluster_inference.yaml
rsync -Pvar single-t4:/home/gcpuser/sky_workdir/output.json /sandbox/output.json
""", filename="inference.sh")

In [ ]:
inference_shell.head()

In [ ]:
upload = sy.ContainerUpload(arg_name="shell_file")
file = sy.ContainerCommandKwarg(name="shell_file", value=upload, required=True, arg_only=True)
inf_kwargs = {
    "shell_file": file,
}

In [ ]:
inference_command = sy.ContainerCommand(
    module_name="blue_book",
    name="inference3",
    image_name="skypilot",
    command="bash",
    args="",
    kwargs=inf_kwargs,
    user_kwargs=["shell_file"],
    user_files=["shell_file", "upload_files"],
    return_filepath="output.json",
    mounts=[azure_key, azure_profile, azure_clouds_config, sky_private_key, sky_public_key]
)

In [ ]:
result = domain_client.api.services.container.add_command(command=inference_command)
result

In [ ]:
print(result.exit_code)
if len(result.jsonstd):
    print(result.jsonstd[0])

In [ ]:
# Cleanup local domain server
node.land()